In [1]:
import cv2 as cv
import numpy as np
import pyautogui

In [2]:
osu_map = "osu_1.mp4"

In [3]:
def mask(img):
    polygons = np.array([(300, 1195), (1700, 1195), (1700, 50), (300, 50)])
    mask = np.zeros_like(img)
    cv.fillPoly(mask, np.array([polygons], dtype=np.int64), 1024)
    masked_img = cv.bitwise_and(img, mask)
    return masked_img

In [4]:
def canny(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2BGRA)
    blur = cv.GaussianBlur(img, (5, 5), 0)
    return cv.Canny(blur, 50, 150)

In [5]:
video = cv.VideoCapture(osu_map)

In [6]:
cv.waitKey(1)

-1

In [7]:
resolution = (1920, 1080)
codec = cv.VideoWriter_fourcc(*"XVID")
filename = "Recording.avi"    
fps = 60.0
out = cv.VideoWriter(filename, codec, fps, resolution)
cv.namedWindow("Live", cv.WINDOW_NORMAL)    
cv.resizeWindow("Live", 1000, 720)

In [8]:
while video.isOpened():
    _, frame = video.read()
    
    img = pyautogui.screenshot()
    frame = np.array(img)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    out.write(frame)
    
#     cv.namedWindow("Video", cv.WINDOW_NORMAL)
#     cv.resizeWindow("Video", 1240, 680)

#     frame = mask(out)
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray_blurred = cv.GaussianBlur(gray, (7, 7), 0)
    
    detected_circles = cv.HoughCircles(gray_blurred, 
                   cv.HOUGH_GRADIENT, 1, 20, param1 = 50,
               param2 = 50, minRadius = 10, maxRadius = 100)
    
    if detected_circles is not None:
        detected_circles = np.uint16(np.around(detected_circles))
        for pt in detected_circles[0, :]:
            a, b, r = pt[0], pt[1], pt[2]
            cv.circle(frame, (a, b), r, (0, 255, 0), 10)
            cv.circle(frame, (a, b), 1, (0, 0, 255), 20)
            cv.imshow("Live", frame)
            
    if cv.waitKey(1) & 0xFF == ord('q'):
        video.release()
        cv.destroyAllWindows()
        